# Libraries installation

In [5]:
pip install numpy folium pyproj plotly geopy

# Localization logic

In [4]:
import numpy as np
import folium
import plotly.graph_objects as go
from pyproj import Transformer

# ===== INPUTS =====
observer_lat = 31.963158
observer_lon = 35.930359
observer_alt = 20  # meters

pitch_deg = -10   # camera pitch (negative = down)
yaw_deg = 45      # yaw degrees from North (0 = North, 90 = East)
target_distance = 100  # meters (max distance to tank)

# ===== COMPUTE DIRECTION VECTOR (ENU) =====
pitch = np.radians(pitch_deg)
yaw = np.radians(yaw_deg)

dx = np.cos(pitch) * np.sin(yaw)   # East component
dy = np.cos(pitch) * np.cos(yaw)   # North component
dz = np.sin(pitch)                  # Up component

direction_vector = np.array([dx, dy, dz])
direction_vector /= np.linalg.norm(direction_vector)

# Scale so that line intersects ground (up=0)
scale = -observer_alt / dz
scale = min(scale, target_distance)

enu_target = direction_vector * scale
east_offset, north_offset, up_offset = enu_target

# ===== COORDINATE TRANSFORMATIONS =====
ecef_from_llh = Transformer.from_crs("epsg:4326", "epsg:4978", always_xy=True)
llh_from_ecef = Transformer.from_crs("epsg:4978", "epsg:4326", always_xy=True)

x0, y0, z0 = ecef_from_llh.transform(observer_lon, observer_lat, observer_alt)
origin_ecef = np.array([x0, y0, z0])

lat_rad = np.radians(observer_lat)
lon_rad = np.radians(observer_lon)

east = np.array([-np.sin(lon_rad), np.cos(lon_rad), 0])
north = np.array([
    -np.sin(lat_rad)*np.cos(lon_rad),
    -np.sin(lat_rad)*np.sin(lon_rad),
    np.cos(lat_rad)
])
up = np.array([
    np.cos(lat_rad)*np.cos(lon_rad),
    np.cos(lat_rad)*np.sin(lon_rad),
    np.sin(lat_rad)
])

ecef_target = origin_ecef + east_offset*east + north_offset*north + up_offset*up
target_lon, target_lat, target_alt = llh_from_ecef.transform(*ecef_target)

print(f"Estimated tank position: lat={target_lat:.6f}, lon={target_lon:.6f}, alt={target_alt:.2f}")

# ===== CREATE FOLIUM MAP =====
m = folium.Map(location=[observer_lat, observer_lon], zoom_start=17)

folium.Marker(
    [observer_lat, observer_lon],
    popup="Camera",
    icon=folium.Icon(color='blue')
).add_to(m)

folium.Marker(
    [target_lat, target_lon],
    popup="Tank",
    icon=folium.Icon(color='red')
).add_to(m)

folium.PolyLine(
    locations=[[observer_lat, observer_lon], [target_lat, target_lon]],
    color='green',
    weight=3
).add_to(m)

m.save("map.html")
print("Folium map saved to map.html")

# ===== CREATE PLOTLY 3D PLOT =====
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=[x0], y=[y0], z=[z0],
    mode='markers',
    marker=dict(size=6, color='blue'),
    name='Camera'
))

fig.add_trace(go.Scatter3d(
    x=[ecef_target[0]], y=[ecef_target[1]], z=[ecef_target[2]],
    mode='markers',
    marker=dict(size=6, color='red'),
    name='Tank'
))

fig.add_trace(go.Scatter3d(
    x=[x0, ecef_target[0]],
    y=[y0, ecef_target[1]],
    z=[z0, ecef_target[2]],
    mode='lines',
    line=dict(color='green', width=3),
    name='Line of Sight'
))

fig.update_layout(
    title="3D ECEF Tank Localization",
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Z (m)',
        aspectmode='data'
    )
)

fig.write_html("plot.html", auto_open=False)
print("Plotly 3D plot saved to plot.html")
print("Open both map.html and plot.html in your browser to view results.")

Estimated tank position: lat=31.963786, lon=35.931096, alt=2.64
Folium map saved to map.html
Plotly 3D plot saved to plot.html
Open both map.html and plot.html in your browser to view results.


# Evaluation

In [8]:
import numpy as np
from pyproj import Transformer
from geopy.distance import geodesic

# ===== LOCALIZATION FUNCTION =====
def localize_target(
    observer_lat, observer_lon, observer_alt,
    pitch_deg, yaw_deg, target_distance
):
    pitch = np.radians(pitch_deg)
    yaw = np.radians(yaw_deg)

    dx = np.cos(pitch) * np.sin(yaw)
    dy = np.cos(pitch) * np.cos(yaw)
    dz = np.sin(pitch)

    direction_vector = np.array([dx, dy, dz])
    direction_vector /= np.linalg.norm(direction_vector)

    scale = -observer_alt / dz
    scale = min(scale, target_distance)

    east_offset = direction_vector[0] * scale
    north_offset = direction_vector[1] * scale
    up_offset = direction_vector[2] * scale

    # Coordinate transformations
    ecef_from_llh = Transformer.from_crs("epsg:4326", "epsg:4978", always_xy=True)
    llh_from_ecef = Transformer.from_crs("epsg:4978", "epsg:4326", always_xy=True)

    x0, y0, z0 = ecef_from_llh.transform(observer_lon, observer_lat, observer_alt)
    origin_ecef = np.array([x0, y0, z0])

    lat_rad = np.radians(observer_lat)
    lon_rad = np.radians(observer_lon)

    east = np.array([-np.sin(lon_rad), np.cos(lon_rad), 0])
    north = np.array([
        -np.sin(lat_rad) * np.cos(lon_rad),
        -np.sin(lat_rad) * np.sin(lon_rad),
        np.cos(lat_rad)
    ])
    up = np.array([
        np.cos(lat_rad) * np.cos(lon_rad),
        np.cos(lat_rad) * np.sin(lon_rad),
        np.sin(lat_rad)
    ])

    ecef_target = origin_ecef + east_offset * east + north_offset * north + up_offset * up
    target_lon, target_lat, target_alt = llh_from_ecef.transform(*ecef_target)

    return target_lat, target_lon, target_alt


# ===== TEST CASES (10 Cases) =====
test_cases = [
    {
        'observer_lat': 31.963158,
        'observer_lon': 35.930359,
        'observer_alt': 20,
        'pitch_deg': -10,
        'yaw_deg': 45,
        'target_distance': 100,
        'true_lat': 31.963300,
        'true_lon': 35.930500,
        'true_alt': 0,
        'description': 'Standard case'
    },
    {
        'observer_lat': 31.963158,
        'observer_lon': 35.930359,
        'observer_alt': 50,
        'pitch_deg': -15,
        'yaw_deg': 90,
        'target_distance': 200,
        'true_lat': 31.963158,
        'true_lon': 35.932500,
        'true_alt': 0,
        'description': 'Looking East, longer distance'
    },
    {
        'observer_lat': 31.963158,
        'observer_lon': 35.930359,
        'observer_alt': 10,
        'pitch_deg': -5,
        'yaw_deg': 180,
        'target_distance': 50,
        'true_lat': 31.962800,
        'true_lon': 35.930359,
        'true_alt': 0,
        'description': 'Looking South, short range'
    },
    {
        'observer_lat': 31.963158,
        'observer_lon': 35.930359,
        'observer_alt': 30,
        'pitch_deg': -12,
        'yaw_deg': 135,
        'target_distance': 150,
        'true_lat': 31.962950,
        'true_lon': 35.931700,
        'true_alt': 0,
        'description': 'Diagonal SW'
    },
    {
        'observer_lat': 31.963158,
        'observer_lon': 35.930359,
        'observer_alt': 25,
        'pitch_deg': -8,
        'yaw_deg': 270,
        'target_distance': 80,
        'true_lat': 31.963158,
        'true_lon': 35.929500,
        'true_alt': 0,
        'description': 'Looking West'
    },
    {
        'observer_lat': 31.963158,
        'observer_lon': 35.930359,
        'observer_alt': 40,
        'pitch_deg': -6,
        'yaw_deg': 0,
        'target_distance': 120,
        'true_lat': 31.964200,
        'true_lon': 35.930359,
        'true_alt': 0,
        'description': 'Looking North'
    },
    {
        'observer_lat': 31.963158,
        'observer_lon': 35.930359,
        'observer_alt': 15,
        'pitch_deg': -14,
        'yaw_deg': 180,
        'target_distance': 60,
        'true_lat': 31.962500,
        'true_lon': 35.930359,
        'true_alt': 0,
        'description': 'Looking South'
    },
    {
        'observer_lat': 31.963158,
        'observer_lon': 35.930359,
        'observer_alt': 20,
        'pitch_deg': -9,
        'yaw_deg': 225,
        'target_distance': 90,
        'true_lat': 31.962700,
        'true_lon': 35.929900,
        'true_alt': 0,
        'description': 'Southwest'
    },
    {
        'observer_lat': 31.963158,
        'observer_lon': 35.930359,
        'observer_alt': 20,
        'pitch_deg': -7,
        'yaw_deg': 315,
        'target_distance': 70,
        'true_lat': 31.963158,
        'true_lon': 35.929700,
        'true_alt': 0,
        'description': 'Northwest'
    },
    {
        'observer_lat': 31.963158,
        'observer_lon': 35.930359,
        'observer_alt': 20,
        'pitch_deg': -11,
        'yaw_deg': 60,
        'target_distance': 110,
        'true_lat': 31.963450,
        'true_lon': 35.930750,
        'true_alt': 0,
        'description': 'Northeast'
    }
]

# ===== EVALUATION LOOP =====
horizontal_errors = []
altitude_errors = []

for i, case in enumerate(test_cases):
    print(f"\nRunning Test Case {i+1}: {case['description']}")

    pred_lat, pred_lon, pred_alt = localize_target(**{
        k: v for k, v in case.items()
        if k not in ['true_lat', 'true_lon', 'true_alt', 'description']
    })

    true_coord = (case['true_lat'], case['true_lon'])
    pred_coord = (pred_lat, pred_lon)
    h_error = geodesic(true_coord, pred_coord).meters
    a_error = abs(case['true_alt'] - pred_alt)

    horizontal_errors.append(h_error)
    altitude_errors.append(a_error)

    print(f"Predicted: ({pred_lat:.6f}, {pred_lon:.6f}, {pred_alt:.2f})")
    print(f"True:      ({case['true_lat']:.6f}, {case['true_lon']:.6f}, {case['true_alt']:.2f})")
    print(f"Horizontal Error: {h_error:.2f} m")
    print(f"Altitude Error: {a_error:.2f} m")

# ===== AVERAGE ERROR =====
avg_horizontal_error = np.mean(horizontal_errors)
avg_altitude_error = np.mean(altitude_errors)

print("\n===== FINAL AVERAGE ERRORS =====")
print(f"Average Horizontal Error: {avg_horizontal_error:.2f} meters")
print(f"Average Altitude Error:   {avg_altitude_error:.2f} meters")


Running Test Case 1: Standard case
Predicted: (31.963786, 35.931096, 2.64)
True:      (31.963300, 35.930500, 0.00)
Horizontal Error: 77.94 m
Altitude Error: 2.64 m

Running Test Case 2: Looking East, longer distance
Predicted: (31.963158, 35.932333, 0.00)
True:      (31.963158, 35.932500, 0.00)
Horizontal Error: 15.79 m
Altitude Error: 0.00 m

Running Test Case 3: Looking South, short range
Predicted: (31.962709, 35.930359, 5.64)
True:      (31.962800, 35.930359, 0.00)
Horizontal Error: 10.11 m
Altitude Error: 5.64 m

Running Test Case 4: Diagonal SW
Predicted: (31.962258, 35.931415, 0.00)
True:      (31.962950, 35.931700, 0.00)
Horizontal Error: 81.34 m
Altitude Error: 0.00 m

Running Test Case 5: Looking West
Predicted: (31.963158, 35.929521, 13.87)
True:      (31.963158, 35.929500, 0.00)
Horizontal Error: 1.98 m
Altitude Error: 13.87 m

Running Test Case 6: Looking North
Predicted: (31.964234, 35.930359, 27.46)
True:      (31.964200, 35.930359, 0.00)
Horizontal Error: 3.80 m
Altitu